In [ ]:
import os
import re
import fitz  # PyMuPDF
import pandas as pd
import numpy as np
from tqdm import tqdm

# Dicionário de palavras-chave organizado por categoria fazer palavras chave gerais e filtrar casos depois e no fim ajustar palavras chave
keywords_dict = {#aquaticos
    'Geral': ['aquátic', 'mar', 'oceano', 'oceânico', 'marinh', 'costa', 'costeir', 'litoral', 'litorâne', #talvez tirar aventoacento e juntar oceanico e coateiro
              'maremoto', 'marítim', 'maritimidade', 'tsunami', 'ultramarina', 'salgad', 'salobra', 'costeira'],
    'Ecossistemas': ['praia', 'mangue', 'marisma', 'grama marinha', 'mar profundo',
                     'fundo do mar', 'coral', 'costão', 'restinga'],
    'Costa': ['mar aberto', 'baía', 'enseada', 'arquipélago', 'brisa', 'ilha'],
    'Organismos': ['plâncton', 'nécton', 'bentos', 'alga', 'peixe', 'crustáceo', 'tubarão', 'baleia', 'foca',
                   'orca', 'golfinho', 'gaivota', 'fragata', 'ave marinha'],
    'Navegação': ['barco', 'barca', 'balsa', 'caravela', 'navio', 'transatlântico', 'cruzeiro', 'submarino',
                  'naufrágio', 'jet-ski', 'lancha', 'píer'],
    'Alimentação': ['pesca', 'pesqueir', 'marisco', 'marisqueir','frutos do mar', 'caranguejo', 'mexilh',
                    'camarão', 'tatuí', 'polvo', 'molusco'],
    'Lazer': ['protetor solar', 'filtro solar', 'canga', 'surf', 'mergulho', 'concha', 'paddle'],
    'Impactos': ['esgoto', 'plástico', 'petróleo', 'aquecimento global', 'mudança climática', 'branqueamento',
                 'acidificação', 'eutrofização', 'poluição', 'mudanças climáticas'],
    'Economia': ['offshore', 'eólica', 'aquacultura', 'economia azul', 'pedras preciosas', 'calcário',
                 'porto', 'indústria naval'],
    'Ciência': ['oceanografia', 'oceanologia'],
    'Cultura': ['deuses', 'lendas', 'monstros', 'Iemanjá', 'Poseidon'],
    'Cinema': ['Nemo', 'Dory', 'sereia', 'Ariel', 'pirata', 'Bob Esponja', 'Luca', 'Aquaman', 'Sammy', 'Willy']
}

In [ ]:

# Caminho base onde estão os PDFs
diretorio_base = "C:/Users/mathe/OneDrive/Documentos/proj_franco/livros"#Ensino Fundamental II"

def encontrar_pdfs(diretorio):
    pdfs = []
    for root, _, files in os.walk(diretorio):
        for file in files:
            if file.endswith('.pdf'):
                pdfs.append(os.path.join(root, file))
    return pdfs

#percorre diretórios
def extrair_ocorrencias(path):
    book = fitz.open(path)
    book_name = os.path.basename(path)
    ocorrencias = []

    for page in book:
        #extraindo os blocos que formam uma pagina
        text_blocks = page.get_text("blocks")
        for bloco in text_blocks:

            #pegando a parte que contém o texto e trata quebras de linha
            texto = re.sub(r'-\n', '', bloco[4])
            texto = re.sub(r'\n', ' ', texto)
            texto = re.sub(r' {2,}', ' ', texto)

            texto_lower = texto.lower()

            #precisa ser dict e nao compiled keywords
            for category, words in keywords_dict.items():
                for word in words:
                    #verifica se há palavras chave dentro do texto do bloco
                    if re.search(word, texto_lower):
                        #caso haja, adicionar à lista de ocorrências
                        ocorrencias.append([
                            book_name,
                            page.number + 1,
                            category,
                            word,  # remove \b...\b
                            texto
                        ])
    return pd.DataFrame(ocorrencias, columns=['Livro', 'Página', 'Categoria', 'Palavra', 'Trecho'])

def remove_caracteres_invalidos(valor):
    if isinstance(valor, str):
        return re.sub(r"[\x00-\x1F]", "", valor)
    return valor


In [ ]:
# Processamento principal
pdfs = encontrar_pdfs(diretorio_base)
df = pd.DataFrame(columns=['Livro', 'Página', 'Categoria', 'Palavra', 'Trecho'])

for pdf in tqdm(pdfs, desc="Processando PDFs"):
    df = pd.concat([df, extrair_ocorrencias(pdf)], ignore_index=True)

In [ ]:
# Lista de palavras que precisam verificação exata no trecho - código para reomver quando palavras como "forquilha" aparecem
palavras_verificadas = ['mar', 'ilha']

# Para cada palavra, filtra as linhas onde:
# - a palavra está na coluna 'palavra'
# - e 'trecho' contém essa palavra como palavra isolada
mascaras = []

for palavra in palavras_verificadas:
    mascara = (df['Palavra'] == palavra) & (df['Trecho'].str.contains(rf'\b{palavra}\b', case=False, regex=True))
    mascaras.append(mascara)

# Junta todas as máscaras com OR (|), e adiciona as linhas que têm outras palavras
mascara_final = pd.concat(mascaras, axis=1).any(axis=1) | ~df['Palavra'].isin(palavras_verificadas)

# Aplica no DataFrame original
df = df[mascara_final]


In [ ]:
#Criar dicionário para substituir as palavras incompletas- palavras que poderiam aparece com gêneros masc. fem. singular e plural

update_keywords = {'aquátic': 'aquático', 'marinh': 'marinho', 'costeir':'costeiro', 'litorâne':'litorâneo', 'marítim': 'marítima',
                   'salgad':'salgada', 'pesqueir':'pesqueira', 'marisqueir':'marisqueira',  'mexilh': 'mexilhão'    
}

df['Palavra'] = df['Palavra'].replace(update_keywords)


#em algum momento a criação do df_final se perdeu, criando de novo aqui rs
df_final = df

In [ ]:
# Filtrando ruídos - análise manual mostrou que essas polissemias ocorriam com grande frequência
remover = ['Costa', 'COSTA', 'mar. 2', 'Porto Alegre', 'Coral', 'canto coral', 'PORTO RICO', 'Porto Rico', 'Sthar Mar']

mascara_ruido = df_final['Trecho'].str.contains('|'.join(remover), case=False, na=False)
mascara_coral_arte = df_final['Trecho'].str.contains('coral', case=False, na=False) & df_final['Livro'].str.contains('arte', case=False, na=False)

df_final = df_final[~(mascara_ruido | mascara_coral_arte)]

# Removendo caracteres inválidos
#df_final2 = df_final.applymap(remove_caracteres_invalidos)
df_final2 = df_final.map(remove_caracteres_invalidos)

# Salvando resultado
output_path = "C:/Users/mathe/OneDrive/Documentos/proj_franco/ocorrencia_palavras_final.xlsx"
df_final2.to_excel(output_path, sheet_name='aba1', index=False)

print("✅ Arquivo gerado com sucesso:", output_path)